In [1]:
!pip install ../input/timm-0-3-2/timm-0.3.2-py3-none-any.whl

Processing /kaggle/input/timm-0-3-2/timm-0.3.2-py3-none-any.whl
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn as optim
import timm
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from PIL import Image
import cv2
fold = 1

In [3]:
df_train = pd.read_csv('../input/cassava-merged-data/train.csv')
df_train.head()

,image_id,label,source
0,1000015157.jpg,0,2020
1,1000201771.jpg,3,2020
2,100042118.jpg,1,2020
3,1000723321.jpg,1,2020
4,1000812911.jpg,3,2020


In [4]:
from sklearn.model_selection import GroupKFold, StratifiedKFold
skf = StratifiedKFold(5, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['label'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

,image_id,label,source,fold
0,1000015157.jpg,0,2020,4
1,1000201771.jpg,3,2020,4
2,100042118.jpg,1,2020,1
3,1000723321.jpg,1,2020,2
4,1000812911.jpg,3,2020,2


In [5]:
df_train.drop(columns = ['source'],inplace=True)

In [6]:
class configure:

    epochs=6

    #   model_name='resnext50_32x4d'
    
    target_columns = [0,1,2,3,4]

    model_name = 'resnext50_32x4d'

    size=512

    batch_size=32

    Train_path = '../input/cassava-merged-data/train_images'

    target_size=5

    device=torch.device('cuda')

    num_workers = 4 

    Test_path =  '../input/ranzcr-clip-catheter-line-classification/test' 


In [7]:
class datasets(Dataset):

    def __init__(self,df, mode='Train'):
        super(datasets,self).__init__()
        self.df = df 
        self.image_name = df['image_id'].values

        self.labels = df[configure.target_columns].values

        self.mode=mode

        self.train_transformer = transforms.Compose([
                transforms.Resize((configure.size,configure.size)),
                transforms.RandomHorizontalFlip(0.5),
                transforms.RandomVerticalFlip(0.5),
                transforms.RandomAffine(0.5),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.5),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        self.val_transformer = transforms.Compose([
                transforms.Resize((configure.size,configure.size)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

    def __len__(self):

        return len(self.df)

    def set_use_cache(self, use_cache):
        self.use_cache = use_cache

    def __getitem__(self,idx):
        image_name= self.image_name[idx]

        image_path= f'{configure.Train_path}/{image_name}'

        image = cv2.imread(image_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image = Image.fromarray(image.astype('uint8'))

        if self.mode =='Train':

            image = self.train_transformer(image)

        else:
            image = self.val_transformer(image)


        label = torch.tensor(self.labels[idx]).float()

        return image,label    

In [8]:
model = timm.create_model(configure.model_name,pretrained=True)
n_features = model.fc.in_features
model.fc = nn.Linear(n_features, configure.target_size)
model = model.cuda()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnext50_32x4d_ra-d733960d.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_ra-d733960d.pth


In [9]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
df_train[[0,1,2,3,4]]= lb.fit_transform(df_train['label'])

In [10]:
train = df_train[df_train['fold'] != fold]

In [11]:
train_datset=datasets(train,mode='Train')
train_loader = torch.utils.data.DataLoader(train_datset,batch_size=20,shuffle=True,num_workers=configure.num_workers)

In [12]:
from torch.autograd import Variable

In [13]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [14]:
from tqdm.notebook import tqdm
model.train()
for epoch in tqdm(range(configure.epochs)):

    for images,labels in tqdm(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        output = model(images)
        labels = labels.type_as(output)
        optimizer.zero_grad()
        
        loss = criterion(output,labels)

        loss.backward()

        optimizer.step()
    exp_lr_scheduler.step()
    print ('Epoch [{}/{}], Loss: {:.4f}'
            .format(epoch+1, configure.epochs, loss.item()))
    if epoch == 0:
        train_loader.dataset.set_use_cache(True)


Epoch [1/6], Loss: 0.1867



Epoch [2/6], Loss: 0.1774



Epoch [3/6], Loss: 0.2145



Epoch [4/6], Loss: 0.1858



Epoch [5/6], Loss: 0.2745



Epoch [6/6], Loss: 0.2011



In [15]:
torch.save(model, 'model.pkl')